In [ ]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import inspect

In [ ]:
# Load and read weekly earnings data file, store into pandas dataframe, look at head
# 1. Weekly 
weekly_earnings_csv = pd.read_csv("Resources/weekly_earnings.csv")
weekly_earnings_df = pd.DataFrame(weekly_earnings_csv)
weekly_earnings_df.head(10)

In [ ]:
# Load and read Labour Force data file, store into pandas dataframe, look at head
#2. labour force
labour_force_csv = pd.read_csv("Resources/labour_force.csv")
labour_force_df = pd.DataFrame(labour_force_csv)
labour_force_df.head(10)

In [ ]:
# Load and read retail turnover data file, store into pandas dataframe, look at head
#3. retail turnover
retail_turnover_csv = pd.read_csv("Resources/retail_turnover.csv")
retail_turnover_df = pd.DataFrame(retail_turnover_csv)
retail_turnover_df.head(10)

In [ ]:
# Clean weekly earnings dataset (units are $ dollars)
#Remove unneccesary rows (0-8, second column, 5th column, 8th column)
weekly_earnings_cleaned = weekly_earnings_df.drop(weekly_earnings_df.index[0:9])
weekly_earnings_cleaned.head()

In [ ]:
#Remove 2nd, 5th, 8th columns
weekly_earnings_reduced = weekly_earnings_cleaned.drop(weekly_earnings_cleaned.columns[[1, 4, 7]], axis = 1)
weekly_earnings_reduced.head()

In [ ]:
#check what the current headings are for columns
weekly_earnings_reduced.columns

In [ ]:
#Renaming Weekly Earnings Headings to match SQL tables

weekly_earnings_reduced.columns = ['Date', 'Male_Earnings_Full_Time_$', 'Male_Earnings_Total_$', 
                                   'Female_Earnings_Full_Time_$', 'Female_Earnings_Total_$', 
                                   'People_Earnings_Full_Time_$', 'People_Earnings_Total_$']
weekly_earnings_reduced.head()

In [ ]:
labour_force_df.head(10)

In [ ]:
# remove unwanted rows (0-9)
labour_force_cleaned = labour_force_df.drop(labour_force_df.index[0:9])
labour_force_cleaned.head()

In [ ]:
#keep certain columns(0, 2, 5, 8, 11, 14, 17) drop the rest
labour_force_reduced = labour_force_cleaned.iloc[:, [0,2,5,8,11,14,17]]
labour_force_reduced.head()

In [ ]:
# rename the columns
labour_force_reduced.columns

In [ ]:
#rename columns to match SQL table
labour_force_reduced.columns = ['Date', 'Employed_Total_Persons_thousands', 'Employed_Total_Males_thousands', 'Employed_Total_Females_thousands', 
                                'Employed_Full_Time_Persons_thousands', 'Employed_Full_Time_Males_thousands', 'Employed_Full_Time_Females_thousands']
labour_force_reduced.head()

In [ ]:
retail_turnover_df.head(10)

In [ ]:
# remove unwanted rows (0-9)
retail_turnover_cleaned = retail_turnover_df.drop(retail_turnover_df.index[0:9])
retail_turnover_cleaned.head()

In [ ]:
#Filter data to keep two columns(0, 14)
retail_turnover_reduced = retail_turnover_cleaned.iloc[:, [0,14]]
retail_turnover_reduced.head()

In [ ]:
#rename headers
retail_turnover_reduced.columns = ['Date', 'Retail_Turnover_WA_$millions']
retail_turnover_reduced.head()

In [ ]:
retail_turnover_reduced.dtypes

In [ ]:
df = pd.merge(weekly_earnings_reduced, labour_force_reduced, on='Date', how='inner')
df


In [ ]:
df.count()

In [ ]:
final_df = pd.merge(df, retail_turnover_reduced, on='Date', how='inner')
final_df.head()

In [ ]:
final_df.dtypes

In [ ]:
final_df.columns

In [ ]:
df2 = final_df.astype({'Date':'datetime64[ns]',
                       'Male_Earnings_Full_Time_$':'float64', 'Male_Earnings_Total_$':'float64',
                       'Female_Earnings_Full_Time_$':'float64', 'Female_Earnings_Total_$':'float64',
                       'People_Earnings_Full_Time_$':'float64', 'People_Earnings_Total_$':'float64',
                       'Employed_Total_Persons_thousands':'float64', 'Employed_Total_Males_thousands':'float64',
                       'Employed_Total_Females_thousands':'float64','Employed_Full_Time_Persons_thousands':'float64',
                       'Employed_Full_Time_Males_thousands':'float64','Employed_Full_Time_Females_thousands':'float64',
                       'Retail_Turnover_WA_$millions':'float64'})
df2.dtypes     

In [ ]:
weekly_earnings_df2 = weekly_earnings_reduced.astype({'Date':'datetime64[ns]',
                       'Male_Earnings_Full_Time_$':'float64', 'Male_Earnings_Total_$':'float64',
                       'Female_Earnings_Full_Time_$':'float64', 'Female_Earnings_Total_$':'float64',
                       'People_Earnings_Full_Time_$':'float64', 'People_Earnings_Total_$':'float64'})
weekly_earnings_df2.dtypes

In [ ]:
labour_force_df2 = labour_force_reduced.astype({'Date':'datetime64[ns]','Employed_Total_Persons_thousands':'float64', 'Employed_Total_Males_thousands':'float64',
                       'Employed_Total_Females_thousands':'float64','Employed_Full_Time_Persons_thousands':'float64',
                       'Employed_Full_Time_Males_thousands':'float64','Employed_Full_Time_Females_thousands':'float64'})
labour_force_df2.dtypes

In [ ]:
retail_turnover_df2 = retail_turnover_reduced.astype({'Date':'datetime64[ns]','Retail_Turnover_WA_$millions':'float64'})
retail_turnover_df2.dtypes

## Load

In [ ]:
df2.head()

In [ ]:
# Save cleaned individual dataframes and final dataframe as csv files
weekly_earnings_df2.to_csv('Output/weekly_earnings_output.csv') 
labour_force_df2.to_csv('Output/labour_force_output.csv') 
retail_turnover_df2.to_csv('Output/retail_turnover_output.csv') 
df2.to_csv('Output/final_output.csv') 


Connect to local database

In [ ]:
# import password and database name
from config import password, database_name

In [ ]:
protocol = 'postgresql'
username = 'postgres'
# password = 3720
host = 'localhost'
port = 5432
# database_name = 'Final_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)
insp = inspect(engine)

Check tables 

In [ ]:
insp.get_table_names()

In [ ]:

from sqlalchemy.types import Integer, DateTime, Float

In [ ]:
pip install psycopg2

In [ ]:
# error - need to chnage dtypes on tables first. Date & float.
# load weekly earnings dataset into database
weekly_earnings_df2.to_sql('Weekly_Earnings', con=engine, if_exists='append', index=False)

In [ ]:
labour_force_df2.to_sql('Labour_Force', con=engine, if_exists='append', index=False)

In [ ]:
retail_turnover_df2.to_sql('Retail_Turnover', con=engine, if_exists='append', index=False)

In [ ]:
df2.to_sql('Final_Table', con=engine, if_exists='append', index=False)